In [1]:
import pandas as pd
import numpy as np
import datetime as datetime


from pandas                 import DataFrame
from pandas                 import read_csv
from numpy                  import array
from numpy                  import random
from sklearn.linear_model   import LogisticRegression
from sklearn.tree           import DecisionTreeClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.ensemble       import GradientBoostingClassifier
from sklearn.svm            import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.metrics        import accuracy_score
from sklearn.metrics        import auc
from sklearn.metrics        import roc_auc_score
from matplotlib             import pyplot
from pandas                 import Series
from pandas                 import get_dummies
from numpy                  import where
from numpy                  import nan
from scipy.stats.mstats     import winsorize
from sklearn.cluster        import KMeans
from sklearn.cluster        import AgglomerativeClustering
from matplotlib             import pyplot
from mpl_toolkits.mplot3d   import Axes3D 
from ieseg                  import partition
from ieseg                  import roc
from ieseg                  import lift
from ieseg                  import cumulativeResponse
from ieseg                  import cumulativeGains


ModuleNotFoundError: No module named 'ieseg'

In [ ]:
# code to plot fast
pip install plotly_express==0.4.0

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())  # Allow sqldf to access global environment

In [3]:
#Importing Data
account = pd.read_csv('./data_berka/account.asc', sep=';')
card = pd.read_csv('./data_berka/card.asc', sep=';')
client = pd.read_csv('./data_berka/client.asc', sep=';')
disp = pd.read_csv('./data_berka/disp.asc', sep=';')
district = pd.read_csv('./data_berka/district.asc', sep=';')
loan = pd.read_csv('./data_berka/loan.asc', sep=';')
order = pd.read_csv('./data_berka/order.asc', sep=';')
trans = pd.read_csv('./data_berka/trans.asc', sep=';', low_memory=False)

In [4]:
#Account Data Treatment - Date, Types, & Frequency Names  
account['a_year'] = account['date'].transform(lambda bn: int('19' + str(bn)[:2]))
account['a_month'] = account['date'].astype(str).str[2:4].astype(int)
account['a_day'] = account['date'].astype(str).str[-2:].astype(int)
account[['account_id','district_id','date']] = account[['account_id','district_id','date']].astype(str)
account['account_date'] =  pd.to_datetime(account.a_year*10000+account.a_month*100+account.a_day,format='%Y%m%d')
account["frequency"] = account["frequency"].replace('POPLATEK MESICNE', "monthly") 
account["frequency"] = account["frequency"].replace('POPLATEK TYDNE', "weekly") 
account["frequency"] = account["frequency"].replace("POPLATEK PO OBRATU", "after_transaction")
account['account_id'] = account['account_id'].astype(str)
account["account_frequency"] = account["frequency"]
account = account.drop(columns=["a_day","a_year", "a_month","date","frequency"])

In [5]:
#Card Data Treatment - Date, Types, & Names  
card['c_year'] = card['issued'].transform(lambda bn: int('19' + str(bn)[:2]))
card['c_month'] = card['issued'].transform(lambda bn: int(str(bn)[2:4]))
card['c_day'] = card['issued'].transform(lambda bn: int(str(bn)[4:6]))
card['card_date'] =  pd.to_datetime(card.c_year*10000+card.c_month*100+card.c_day,format='%Y%m%d')
card["type"] = card["type"]
card = card.drop(columns=["c_day","c_year", "c_month","issued"])
card = card.rename(columns={"type": "card_type"})

In [6]:
#Client Data Treatment - Birth Year, Dates, Age, Age Group, Types, & Frequency Names 
client['b_year'] = client['birth_number'].transform(lambda bn: int('19' + str(bn)[:2]))
client['b_day'] = client['birth_number'].astype(str).str[-2:].astype(int)
client['age'] = 1999 - client['b_year']
client['age_group'] = client['age'] // 10 * 10
client['client_id'] = client['client_id'].astype(str)
client['district_id'] = client['district_id'].astype(str)

In [7]:
#Function to add gender and change birth 
def to_month_gender(birth_number):
    s = str(birth_number)
    birth_month = int(s[2:4])
    if birth_month > 50:
        gender = "F"
        birth_month = birth_month - 50
    else:
        gender = 'M'
    return pd.Series({'birth_month':birth_month, 'gender':gender})
client[['b_month', 'gender']] = client['birth_number'].apply(to_month_gender)

In [8]:
# Create dob
client['client_dob'] =  pd.to_datetime(client.b_year*10000+client.b_month*100+client.b_day,format='%Y%m%d')
# Drop columns from client
client = client.drop(columns=["b_day","b_year", "b_month","birth_number"])

In [9]:
#Transaction Data Treatment - Date, Translation, & Columns
trans['t_year'] = 1900 + trans['date'].astype(str).str[:2].astype(int)
trans['t_year'] = trans['t_year'].astype(int)
trans['t_month'] = trans['date'].astype(str).str[2:4].astype(int)
trans['t_day'] = trans['date'].astype(str).str[-2:].astype(int)
trans['trans_date'] =  pd.to_datetime(trans.t_year*10000+trans.t_month*100+trans.t_day,format='%Y%m%d')

In [10]:
trans["trans_op"] = trans["operation"].replace({nan: "t_miss", "VKLAD" : "t_cc", "PREVOD Z UCTU" : "t_coll_bank", "VYBER" : "to_wd_cash", "PREVOD NA UCET" : "t_remm_bank", "VYBER KARTOU":"t_cc_wd"})
trans["trans_type"] = trans["type"].replace({"PRIJEM" : "t_credit", "VYDAJ" : "t_wd", "VYBER" : "ty_wd_cash"})
trans["k_symbol"] = trans["k_symbol"].replace({"POJISTNE": "t_ins_pmt", "SLUZBY" : "t_pmt_sta", "UROK": "t_int_cr", "SANKC. UROK" : "t_sac_neg_bal", "SIPO": "t_house", "DUCHOD": "t_pension", "UVER": "t_loan_pmt"})
trans.k_symbol = trans.k_symbol.fillna('t_others')
trans.k_symbol[trans.k_symbol == ' '] = 't_others'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [11]:
trans["bank"] = trans["bank"].replace({nan: 0})
trans["account"] = trans["account"].replace({nan: 0})
trans['account_id'] = trans.account_id.astype(str)
trans = trans.rename(columns={"amount": "t_amt", "balance" : "t_bal", "account": "t_acct", "bank": "t_bank", "k_symbol":"t_char"})

In [12]:
trans = trans.drop(columns=["t_day","t_year", "t_month","date", "operation","type","operation"])

In [13]:
#Disposition Data Treatment - Columns Types
disp['client_id'] = disp['client_id'].astype(str)
disp['account_id'] = disp['account_id'].astype(str)
disp["type"] = disp["type"].replace({"OWNER":"Owner","DISPONENT":"Disponent"})
disp = disp.rename(columns={"type": "client_type"})

In [14]:
#Order Data Treatment - Columns Names & Types
order.k_symbol = order.k_symbol.fillna('o_others')
order.k_symbol[order.k_symbol == ' '] = 'o_others'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
order["k_symbol"] = order["k_symbol"].replace({"POJISTNE": "o_ins_pmt", "SIPO": "o_house_pmt", "UVER": "o_loan_pmt", "LEASING": "o_lease"})
order['order_id'] = order['order_id'].astype(str)
order['account_id'] = order['account_id'].astype(str)
order = order.rename(columns={"amount": "o_amt", "k_symbol":"o_char"})

In [16]:
#Loan Data Treatment - Dates, Columns Names & Types
loan['l_year'] = 1900 + loan['date'].astype(str).str[:2].astype(int)
loan['l_year'] = loan['l_year'].astype(int)
loan['l_month'] = loan['date'].astype(str).str[2:4].astype(int)
loan['l_day'] = loan['date'].astype(str).str[-2:].astype(int)
loan['loan_date'] =  pd.to_datetime(loan.l_year*10000+loan.l_month*100+loan.l_day,format='%Y%m%d')
loan["status"] = loan["status"].replace({"A": "CF_OK", "B" : "CF_Debt", "C": "CNF_OK", "D" : "CNF_Debt"})
loan['loan_id'] = loan['loan_id'].astype(str)
loan['account_id'] = loan['account_id'].astype(str)
loan['date'] = loan['date'].astype(str)
loan['amount'] = loan['amount'].astype(float)
loan = loan.rename(columns={"amount": "l_amt", "payments":"l_pmts", "status":"l_status","duration":"l_dur"})

In [17]:
# Drop column "l_day","l_year", "l_month","date" of LOAN
loan = loan.drop(columns=["l_day","l_year", "l_month","date"])

In [18]:
#District Data Treatment - Columns Names & Types
district['A1'] = district['A1'].astype(str)
district['A12'] = pd.to_numeric(district['A12'],errors='coerce')
district['A15'] = pd.to_numeric(district['A15'],errors='coerce')
district.rename(columns = {'A1' : 'district_id', 'A2':'dist_name','A3':'reg_name','A4':'population','A5':'muni_<_499','A6':'muni_500-1999','A7':'muni_2000-9999','A8':'muni_>_10000','A9':'num_cities','A10':'ratio_no_inhab','A11':'avg_salary','A12':'unemploymant_95','A13':'unemploymant_96','A14':'enterpre_1000','A15':'crimes_95','A16':'crimes_96'}, inplace = True)
district.fillna(0, inplace=True)

In [19]:
# Create new columns based on 'trans' and get the total amount spent
trans1 = trans.pivot_table(index = 'account_id', columns = ['trans_op'], values = 't_amt', aggfunc = 'sum')
trans2 = trans.pivot_table(index = 'account_id', columns = ['trans_type'], values = 't_amt', aggfunc = 'sum')
trans3 = trans.pivot_table(index = 'account_id', columns = ['t_char'], values = 't_amt', aggfunc = 'sum')

In [20]:
#keep the categories rows
trans4 = trans[['trans_op','trans_type','t_char','account_id']]
trans4['account_id'] = trans4.account_id.astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
trans1.fillna(0, inplace=True) 
trans1.loc[:,'trans_op_tot'] = trans1.sum(axis=1)

In [22]:
trans2.fillna(0, inplace=True) 
trans2.loc[:,'trans_ty_tot'] = trans2.sum(axis=1)

In [23]:
trans3.fillna(0, inplace=True) 
trans3.loc[:,'trans_char_tot'] = trans3.sum(axis=1)

In [24]:
# Merge on LEFT 'trans1' and 'trans2'
left_table = trans1
right_table = trans2
trans12 = pd.merge(left_table,
                right_table,
                how ='left',
                left_on ='account_id',
                right_on ='account_id'
                )

In [25]:
# Merge on LEFT 'trans12' and 'trans3' and get the new table out of Transactions
left_table = trans12
right_table = trans3
trans123 = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )

In [26]:
trans123.reset_index(inplace=True)
trans123['account_id'] = trans123['account_id'].astype(str)

In [27]:
trans123.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 21 columns):
account_id        4500 non-null object
t_cc              4500 non-null float64
t_cc_wd           4500 non-null float64
t_coll_bank       4500 non-null float64
t_miss            4500 non-null float64
t_remm_bank       4500 non-null float64
to_wd_cash        4500 non-null float64
trans_op_tot      4500 non-null float64
t_credit          4500 non-null float64
t_wd              4500 non-null float64
ty_wd_cash        4500 non-null float64
trans_ty_tot      4500 non-null float64
t_house           4500 non-null float64
t_ins_pmt         4500 non-null float64
t_int_cr          4500 non-null float64
t_loan_pmt        4500 non-null float64
t_others          4500 non-null float64
t_pension         4500 non-null float64
t_pmt_sta         4500 non-null float64
t_sac_neg_bal     4500 non-null float64
trans_char_tot    4500 non-null float64
dtypes: float64(20), object(1)
memory usage: 738.4+ KB

In [31]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())  # Allow sqldf to access global environment

In [32]:
# Select specific columns from district and call it dist
dist = pysqldf("""
SELECT district_id, dist_name, reg_name, avg_salary as avg_dist_sal
FROM district
""")

In [33]:
#Treating Order1 Table
order1 = order.pivot_table(index = 'account_id', columns = ['o_char'], values = 'o_amt', aggfunc = 'sum')
order1.fillna(0, inplace=True)
order1.loc[:,'orders_total'] = order1.sum(axis=1)
order1.reset_index(inplace=True)
order1['account_id'] = order1['account_id'].astype(str)
order1['o_char'] = order.loc[:,'o_char']

In [34]:
#Merge Disp1 & Card
left_table = disp
right_table = card
disp_card = pd.merge(left_table,
                right_table,
                how='left',
                left_on='disp_id',
                right_on='disp_id'
                )

In [35]:
#Merge Client & Disp_card
left_table = client
right_table = disp_card
client_disp_card = pd.merge(left_table,
                right_table,
                how='left',
                left_on='client_id',
                right_on='client_id'
                )

In [36]:
#Merge Account & Trans123
left_table = account
right_table = trans123
account_trans123 = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )

In [37]:
#Merge Account_Trans123 & Loan
left_table = account_trans123
right_table = loan
acct_trans_loan = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )

In [38]:
#Merge acct_trans_loan_order
left_table = acct_trans_loan
right_table = order1
acct_trans_loan_order = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )

In [39]:
#Merge Total Client & Total Account
left_table = client_disp_card
right_table = acct_trans_loan_order
client_account_total = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )

In [40]:
#Merge Client_Account_Total & Ditrict 
left_table = client_account_total
right_table = dist
datamart = pd.merge(left_table,
                right_table,
                how='left',
                left_on='district_id_x',
                right_on='district_id'
                )

In [41]:
# Fill Na with 0
datamart.fillna(0, inplace=True)

In [42]:
# Drop duplicate columns from main table
datamart18 = datamart.drop(columns=["district_id_y","district_id_x"])

In [43]:
datamart18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 50 columns):
client_id            5369 non-null object
age                  5369 non-null int64
age_group            5369 non-null int64
gender               5369 non-null object
client_dob           5369 non-null datetime64[ns]
disp_id              5369 non-null int64
account_id           5369 non-null object
client_type          5369 non-null object
card_id              5369 non-null float64
card_type            5369 non-null object
card_date            5369 non-null object
account_date         5369 non-null datetime64[ns]
account_frequency    5369 non-null object
t_cc                 5369 non-null float64
t_cc_wd              5369 non-null float64
t_coll_bank          5369 non-null float64
t_miss               5369 non-null float64
t_remm_bank          5369 non-null float64
to_wd_cash           5369 non-null float64
trans_op_tot         5369 non-null float64
t_credit             5369 non-nu

In [44]:
# Check columns
for column in datamart18.columns:
    print(datamart18[column].name)

client_id
age
age_group
gender
client_dob
disp_id
account_id
client_type
card_id
card_type
card_date
account_date
account_frequency
t_cc
t_cc_wd
t_coll_bank
t_miss
t_remm_bank
to_wd_cash
trans_op_tot
t_credit
t_wd
ty_wd_cash
trans_ty_tot
t_house
t_ins_pmt
t_int_cr
t_loan_pmt
t_others
t_pension
t_pmt_sta
t_sac_neg_bal
trans_char_tot
loan_id
l_amt
l_dur
l_pmts
l_status
loan_date
o_house_pmt
o_ins_pmt
o_lease
o_loan_pmt
o_others
orders_total
o_char
district_id
dist_name
reg_name
avg_dist_sal


In [48]:
# Download datamart
datamart18.to_csv(pd.read_csv('./data_berka/account.asc', sep=';'))

ValueError: Invalid file path or buffer object type: <class 'pandas.core.frame.DataFrame'>

In [42]:
datamart18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 50 columns):
client_id            5369 non-null object
age                  5369 non-null int64
age_group            5369 non-null int64
gender               5369 non-null object
client_dob           5369 non-null datetime64[ns]
disp_id              5369 non-null int64
account_id           5369 non-null object
client_type          5369 non-null object
card_id              5369 non-null float64
card_type            5369 non-null object
card_date            5369 non-null object
account_date         5369 non-null datetime64[ns]
account_frequency    5369 non-null object
t_cc                 5369 non-null float64
t_cc_wd              5369 non-null float64
t_coll_bank          5369 non-null float64
t_miss               5369 non-null float64
t_remm_bank          5369 non-null float64
to_wd_cash           5369 non-null float64
trans_op_tot         5369 non-null float64
t_credit             5369 non-nu

In [43]:
#Datamart with only owners for analysis 
datamart18_s = datamart18[datamart18.client_type == 'Owner']

In [77]:
# Number of clients per age group
r = DataFrame(datamart18_s.groupby(['reg_name'])['client_id'].nunique())
r_1 =r.rename(columns = {'client_id':'nbr_clients'}, inplace=True)
rr =r.loc[:'nbr_clients'].sum()
r['perc'] = r['nbr_clients']/4500
r.reset_index(inplace=True)
r.sort_values
r.sort_values('nbr_clients',ascending =False)

#Plot the data
fig = px.bar(r, x='reg_name', y='nbr_clients')
print(t,fig.show())

   age_group  nbr_clients      perc
0         10          145  0.032222
1         20          907  0.201556
2         30          804  0.178667
3         40          821  0.182444
4         50          843  0.187333
5         60          517  0.114889
6         70          389  0.086444
7         80           74  0.016444 None


In [76]:
# Number of clients per age group
d = DataFrame(datamart18_s.groupby(['dist_name'])['client_id'].nunique())
d_1 =d.rename(columns = {'client_id':'nbr_clients'}, inplace=True)
dd =d.loc[:'nbr_clients'].sum()
d['perc'] = d['nbr_clients']/4500
d.reset_index(inplace=True)
d.sort_values
d.sort_values('nbr_clients',ascending =False)

#Plot the data
fig = px.bar(d, x='dist_name', y='nbr_clients')
print(t,fig.show())

   age_group  nbr_clients      perc
0         10          145  0.032222
1         20          907  0.201556
2         30          804  0.178667
3         40          821  0.182444
4         50          843  0.187333
5         60          517  0.114889
6         70          389  0.086444
7         80           74  0.016444 None


In [60]:
# Number of clients per age group
t = DataFrame(datamart18_s.groupby(['age_group'])['client_id'].nunique())
t_1 =t.rename(columns = {'client_id':'nbr_clients'}, inplace=True)
tt =t.loc[:'nbr_clients'].sum()
t['perc'] = t['nbr_clients']/4500
t.reset_index(inplace=True)

fig = px.bar(t, x='age_group', y='nbr_clients')
print(t,fig.show())


   age_group  nbr_clients      perc
0         10          145  0.032222
1         20          907  0.201556
2         30          804  0.178667
3         40          821  0.182444
4         50          843  0.187333
5         60          517  0.114889
6         70          389  0.086444
7         80           74  0.016444 None


In [45]:
# Number of clientes per gender and age group 
a_g = datamart18_s.pivot_table(index = 'age_group', columns = ['gender'], values = 'client_id', aggfunc = 'nunique')
a_g['total'] = a_g['F'] + a_g['M']
a_g['Perc F'] = a_g['F']/a_g['total']
a_g['Perc M'] = a_g['M']/a_g['total']
a_g

gender,F,M,total,Perc F,Perc M
age_group,,,,,
10,69,76,145,0.475862,0.524138
20,474,433,907,0.522602,0.477398
30,411,393,804,0.511194,0.488806
40,411,410,821,0.500609,0.499391
50,441,402,843,0.523132,0.476868
60,193,324,517,0.373308,0.626692
70,185,204,389,0.475578,0.524422
80,24,50,74,0.324324,0.675676


In [46]:
# Count of loan_id
lo = datamart18_s
l= lo[lo.loan_id != 0].pivot_table(index = 'age_group', columns = ['gender'], values = 'loan_id', aggfunc = 'nunique')
l['total'] = l['F'] + l['M']
l['Perc F'] = l['F']/l['total']
l['Perc M'] = l['M']/l['total']
l

gender,F,M,total,Perc F,Perc M
age_group,,,,,
10,13,3,16,0.812500,0.187500
20,79,68,147,0.537415,0.462585
30,80,78,158,0.506329,0.493671
40,82,75,157,0.522293,0.477707
50,85,67,152,0.559211,0.440789
60,9,43,52,0.173077,0.826923


In [47]:
# Count of Card_id
ca = datamart18_s
c = ca[ca.card_id != 0].pivot_table(index = 'age_group', columns = ['gender'], values = 'card_id', aggfunc = 'nunique')
c['total'] = c['F'] + c['M']
c['Perc F'] = c['F']/c['total']
c['Perc M'] = c['M']/c['total']
c

gender,F,M,total,Perc F,Perc M
age_group,,,,,
10,26.0,26.0,52.0,0.500000,0.500000
20,107.0,107.0,214.0,0.500000,0.500000
30,84.0,87.0,171.0,0.491228,0.508772
40,82.0,88.0,170.0,0.482353,0.517647
50,110.0,108.0,218.0,0.504587,0.495413
60,9.0,49.0,58.0,0.155172,0.844828
70,5.0,3.0,8.0,0.625000,0.375000
80,NaN,1.0,NaN,NaN,NaN


In [48]:
# Credit Card and non credit card holders and their loan status
l_c = DataFrame(ca[ca.card_id != 0].groupby('l_status')['card_id'].nunique())
l_c_oo = DataFrame(ca[ca.card_id == 0].groupby('l_status')['client_id'].agg('nunique'))
l_c['No_credit_card'] = l_c_oo.loc[:,:]
l_c

,card_id,No_credit_card
l_status,,
0,722,3096
CF_Debt,2,29
CF_OK,60,143
CNF_Debt,3,42
CNF_OK,105,298


In [49]:
# Order - Type
hh =DataFrame(order.groupby('o_char')['o_amt'].agg('sum'))
hh['percentage'] = hh['o_amt']/6257793560.29999
hh

,o_amt,percentage
o_char,,
o_house_pmt,13965417.0,0.002232
o_ins_pmt,686927.0,0.000110
o_lease,759527.1,0.000121
o_loan_pmt,3035184.5,0.000485
o_others,2781938.0,0.000445


In [50]:
# Transactions - Type
mo =DataFrame(trans.groupby('trans_type')['t_amt'].agg('sum'))
mo['percentage'] = mo['t_amt']/6257793560.29999
#Transactions - Operations
aa =DataFrame(trans.groupby('trans_op')['t_amt'].agg('sum'))
aa['percentage'] = aa['t_amt']/6257793560.29999
#Transactions - Characterization
bb =DataFrame(trans.groupby('t_char')['t_amt'].agg('sum'))
bb['percentage'] = bb['t_amt']/6257793560.29999
display(mo,aa, bb )

,t_amt,percentage
trans_type,,
t_credit,3.227472e+09,0.515752
t_wd,2.821717e+09,0.450913
ty_wd_cash,2.086038e+08,0.033335


,t_amt,percentage
trans_op,,
t_cc,2.418522e+09,0.386482
t_cc_wd,1.817040e+07,0.002904
t_coll_bank,7.814800e+08,0.124881
t_miss,2.747069e+07,0.004390
t_remm_bank,6.726378e+08,0.107488
to_wd_cash,2.339513e+09,0.373856


,t_amt,percentage
t_char,,
t_house,4.985064e+08,0.079662
t_ins_pmt,2.417519e+07,0.003863
t_int_cr,2.747069e+07,0.004390
t_loan_pmt,5.525230e+07,0.008829
t_others,5.482193e+09,0.876058
t_pension,1.674721e+08,0.026762
t_pmt_sta,2.686418e+06,0.000429
t_sac_neg_bal,3.766330e+04,0.000006


# Merging experimentation

In [ ]:
# Merge between Client and Disposition = client_disp

In [ ]:
left_table = client
right_table = disp
client_disp = pd.merge(left_table,
                right_table,
                how='left',
                left_on='client_id',
                right_on='client_id'
                )
client_disp

In [ ]:
# Merge between Client_Disp and Account = Client_Disp_Acc

In [ ]:
left_table = client_disp
right_table = account
Client_Disp_Acc = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )
Client_Disp_Acc.head()

In [ ]:
# Merge between Client_Disp_Acc and Transaction = Client_Disp_Acc_trans

In [ ]:
left_table = Client_Disp_Acc
right_table = trans
Client_Disp_Acc_trans = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )
Client_Disp_Acc_trans

In [ ]:
# Joint tables Client - Disposition - Account - Transaction
# Intent to explore the transacctions per account and operations

In [ ]:
a = Client_Disp_Acc_trans.loc[:,['client_id', 'account_id', 'type_y', 'amount', 'balance', 'operation', 'k_symbol']]
a.head()

In [ ]:
a.groupby('operation')['amount'].agg('sum')

In [ ]:
a.pivot_table(index = 'client_id', columns = ['operation'], values = 'amount', aggfunc = 'sum')

In [ ]:
# Intent to explore the transacctions per account and K_symbol

In [ ]:
trans_PT = a.pivot_table(index = 'client_id', columns = ['k_symbol'], values = 'amount', aggfunc = 'sum')

In [ ]:
trans_PT.describe()

In [ ]:
#merge for Credit card, Disposition, and client
#Interest in understanding the distribution of Credit cards

In [ ]:
left_table = client
right_table = disp
client_disp = pd.merge(left_table,
                right_table,
                how='left',
                left_on='client_id',
                right_on='client_id'
                )
client_disp.head()

In [ ]:
left_table = client_disp
right_table = card
client_disp_cc = pd.merge(left_table,
                right_table,
                how='left',
                left_on='disp_id',
                right_on='disp_id'
                )
client_disp_cc.head()

In [ ]:
creditcard_info = client_disp_cc.loc[:,['client_id', 'account_id', 'type_x', 'type_y', 'card_id', 'year_issued', 'gender', 'age_group']]
creditcard_info.head()

In [ ]:
#Interest in the division of credit cards
#Based on this information there is a great opportunity to move customers from classic Credit cards to gold.
creditcard_info.pivot_table(index = 'client_id', columns = ['type_y'], values = 'gender', aggfunc = 'sum').count()

In [ ]:
#Interest in understand if there is a significatuve difference between credit cards per gender.
#Although there is no significant difference, there is an opportunity to offer more credit cards to the customer database.
creditcard_info.pivot_table(index = 'client_id', columns = ['gender'], values = 'type_y', aggfunc = 'count').sum()

In [ ]:
#interest in understanding the distribution of Credit cards among the different group ages of the customer database.
#The highest concentration is between 20's and 50's, for which there is an opportunity to offer in the inner ranges.
creditcard_info.pivot_table(index = 'client_id', columns = ['age_group'], values = 'type_y', aggfunc = 'count').sum()

In [ ]:
# Merge for Client, disposition,loan  
# Interest in understanding the distribution of Credit cards

In [ ]:
left_table = client_disp
right_table = loan
client_disp_loan = pd.merge(left_table,
                right_table,
                how='left',
                left_on='account_id',
                right_on='account_id'
                )
client_disp_loan.head()

In [ ]:
loan_clientinfo = client_disp_loan.loc[:,['client_id', 'type','age_group','duration','amount', 'status', 'gender', 'account_id']]
loan_clientinfo.info()

In [ ]:
# Intent to understand the tipe of customers by loans monetary wise
# Codigo extra: loan_clientinfo.pivot_table(index = 'status', columns = ['client_id'], values = 'amount', aggfunc = 'count').sum()
loan_clientinfo.pivot_table(columns = ['status'], values = 'amount', aggfunc = 'sum')

In [ ]:
# Interest in the distribution of loan pero category loan (dependants and owners are taken into consideration)
loan_clientinfo.pivot_table(columns = ['status'], values = 'account_id', aggfunc = 'count')

In [ ]:
# Interest in the distribution of loan per group age (dependants and owners are taken into consideration)
loan_clientinfo.pivot_table(columns = ['age_group'], values = 'status', aggfunc = 'count')